In [ ]:
!pip install openai faiss-cpu sentence-transformers streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
!pip install openai faiss-cpu sentence-transformers PyMuPDF

In [26]:
!pip install openai==0.28
import os
import openai
import fitz  # PyMuPDF
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from google.colab import files

In [27]:
# Upload PDF
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Extract text
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

raw_text = extract_text_from_pdf(filename)
print("✅ Text Extracted")

Saving Unit-2.pdf to Unit-2 (3).pdf
✅ Text Extracted


In [28]:
# Chunk text
def chunk_text(text, chunk_size=300):
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

chunks = chunk_text(raw_text)
print(f"✅ Total Chunks: {len(chunks)}")

✅ Total Chunks: 17


In [29]:
# Load MiniLM Model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Create embeddings
embeddings = model.encode(chunks)

# Store in FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("✅ Stored all chunks in FAISS")

✅ Stored all chunks in FAISS


In [30]:
# Get user query
user_query = input("❓ Enter your question: ")
query_embedding = model.encode([user_query])

# Search top 3 similar chunks
top_k = 3
D, I = index.search(np.array(query_embedding), top_k)

retrieved_chunks = [chunks[i] for i in I[0]]
context = "\n".join(retrieved_chunks)

print("\n📄 Top Matching Chunks:\n", context)

❓ Enter your question: what is deep learning?

📄 Top Matching Chunks:
 Unit-2 Deep Learning Architectures NRI INSTITUTE OF TECHNOLOGY 1 Unit-2 Deep Learnig Architectures Machine Learning and Deep Learning: Machine learning is a subfield of artificial intelligence that focuses on the development of algorithms and statistical models that enable computers to learn and make predictions or decisions without being explicitly programmed. It involves training algorithms on large datasets to identify patterns and relationships and then using these patterns to make predictions or decisions about new data. What are the Different Types of Machine Learning? Machine learning is further divided into categories based on the data on which we are training our model. • Supervised Learning – This method is used when we have Training data along with the labels for the correct answer. • Unsupervised Learning – In this task our main objective is to find the patterns or groups in the dataset at hand because w

In [31]:
!pip install transformers

In [32]:
from transformers import pipeline

# Load QA model from HuggingFace
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Ask question using top retrieved context
qa_input = {
    'question': user_query,
    'context': context
}

result = qa_pipeline(qa_input)
answer = result['answer']
score = result['score']
print(f"\n🤖 Answer: {answer}")

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



🤖 Answer: a subfield of artificial intelligence


In [33]:
# Trust score = similarity of query and top chunks
from numpy.linalg import norm

def cosine_sim(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

similarities = []
query_vec = query_embedding[0]

for i in I[0]:
    chunk_vec = embeddings[i]
    sim = cosine_sim(query_vec, chunk_vec)
    similarities.append(sim)

trust_score = np.mean(similarities)
print(f"🔐 Trust Score (0 to 1): {round(trust_score, 2)}")

🔐 Trust Score (0 to 1): 0.699999988079071


In [34]:
print("\n==============================")
print(f"QUESTION: {user_query}")
print(f"ANSWER: {answer}")
print(f"MODEL CONFIDENCE: {round(score, 2)}")
print(f"TRUST SCORE: {round(trust_score, 2)}")
print("==============================")


QUESTION: what is deep learning?
ANSWER: a subfield of artificial intelligence
MODEL CONFIDENCE: 0.51
TRUST SCORE: 0.699999988079071
